In [1]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

ANALISI OSPEDALI

Utilizzeremo i seguenti dati: 

1. **Ospedali**


Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

In [4]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_OSPEDALI_CLEAN),
     "tipo": "Ospedali"}
     ]
Graph_functions.auto_analysis_poi(gdf_list,PATH_GEOJSON="../Data/Clean/Analisi/Ospedali.geojson")

In [5]:
gdf_analisi_ospedali = gpd.read_file("../Data/Clean/Analisi/Ospedali.geojson", driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [6]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_ospedali_mod = gdf_analisi_ospedali[gdf_analisi_ospedali["artificial"] != True]

# Eliminiamo i percorsi completamente contenuti all'interno dei "poi" poligonari: parchi
gdf = gpd.read_file(PATH_OSPEDALI_CLEAN)
geometry_parchi = gdf.union_all()
gdf_senza_poi = []
for idx, row in gdf_analisi_ospedali_mod.iterrows():
    if not geometry_parchi.contains(row.geometry):
        gdf_senza_poi.append(row)
gdf_analisi_ospedali_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Salviamo per test di visualizzazione su kepler
gdf_analisi_ospedali_mod.to_file("../Data/Clean/Analisi/ospedali_senza_poi.geojson")

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_ospedali_mod_most_highways = gdf_analisi_ospedali_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [7]:
# Stampiamo il risultato
print(gdf_analisi_ospedali_mod_most_highways)
print(f"Tot: {gdf_analisi_ospedali_mod_most_highways["length"].sum()}")

                           length
highway                          
cycleway                39.189874
footway                 15.456106
tertiary                 8.819226
residential              4.817837
secondary                4.593407
primary                  4.326607
service                  2.566652
path                     0.572301
pedestrian               0.542666
track                    0.516093
unclassified             0.281732
footway steps            0.187148
residential secondary    0.146541
residential pedestrian   0.146171
track service            0.126378
residential path         0.088589
cycleway footway         0.079023
tertiary_link            0.050109
primary_link             0.012722
Tot: 82.51918564502445
